In [71]:
import pandas as pd
import os
import psycopg2
from dotenv import load_dotenv
from psycopg2.extras import execute_values
df = pd.read_csv('/home/ivancampos/Documents/Programming Stuff/AtomicLocks/Data Science/Data/League_players.csv')

In [72]:
df

,PlayerId,FirstName,LastName,CommonName,MatchName,Position,Gender,BirthDate,BirthCountry,Nationality,Updated
0,100000576,Lee,Da-yun,Lee Da-yun,Spirit,Jungle,Male,1996-07-01T00:00:00,Korea Republic,Korea Republic,2021-06-11T08:05:44
1,100000577,Lee,Seo-haeng,Lee Seo-haeng,Kuro,Mid,Male,1994-01-04T00:00:00,Korea Republic,Korea Republic,2021-06-11T08:07:05
2,100000578,Ha,Jong-Hoon,Ha Jong-Hoon,Kramer,ADC,Male,1993-03-09T00:00:00,Korea Republic,Korea Republic,2021-04-04T12:10:01
3,100000579,Bak,Jong-ik,Bak Jong-ik,TusiN,Support,Male,1995-11-12T00:00:00,Korea Republic,Korea Republic,2021-06-11T08:06:56
4,100000580,Lee,Jae-ha,Lee Jae-ha,Mowgli,Jungle,Male,1998-04-14T00:00:00,Korea Republic,Korea Republic,2021-06-11T08:04:05
...,...,...,...,...,...,...,...,...,...,...,...
1733,100003492,Gen,Takahashi,Gen Takahashi,Phantom,Support,Male,NaN,Japan,Japan,2021-06-22T16:53:50
1734,100003493,Sora,Tobita,Sora Tobita,momo,Top,Male,NaN,Japan,Japan,2021-06-22T16:53:50
1735,100003496,Philipp,Englert,Philipp Englert,Lilipp,Support,Male,NaN,Germany,Germany,2021-06-28T08:51:32
1736,100003497,Andrey,Yakovlev,Andrey Yakovlev,Argonavt,ADC,Male,NaN,Ukraine,Ukraine,2021-06-25T14:59:00


In [73]:
df.isnull().sum().sum()

1067

In [74]:
df.fillna('?',inplace=True)

In [75]:
df.to_csv('League_players.csv')

In [76]:
df = pd.read_csv('/home/ivancampos/Documents/Programming Stuff/AtomicLocks/Data Science/notebooks/League_players.csv')

In [77]:
df.dtypes

Unnamed: 0       int64
PlayerId         int64
FirstName       object
LastName        object
CommonName      object
MatchName       object
Position        object
Gender          object
BirthDate       object
BirthCountry    object
Nationality     object
Updated         object
dtype: object

In [78]:
create_LCS_players_table = """
DROP TABLE IF EXISTS lcs_players;
CREATE TABLE lcs_players (
  ID INT,
  PlayerId INT,
  FirstName VARCHAR,
  LastName VARCHAR,
  CommonName VARCHAR,
  MatchName VARCHAR,
  Position VARCHAR,
  Gender VARCHAR,
  BirthDate VARCHAR,
  BirthCountry VARCHAR,
  Nationality VARCHAR,
  Updated VARCHAR
)
"""

In [79]:
load_dotenv("/home/ivancampos/Documents/Programming Stuff/AtomicLocks/Data Science/Data/.env")

DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")


In [80]:
conn = psycopg2.connect(
            dbname= DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT)

cur = conn.cursor()

In [81]:
cur.execute(create_LCS_players_table)
conn.commit()


In [82]:
insert_lcs_players = """
INSERT INTO lcs_players 
(PlayerId,FirstName,LastName,CommonName,MatchName,Position,Gender,BirthDate,BirthCountry,Nationality,Updated)
VALUES""" + str(tuple(row)) + ";"

In [83]:
execute_values(cur, """
    INSERT INTO lcs_players
    (ID,PlayerId,FirstName,LastName,CommonName,MatchName,Position,Gender,BirthDate,BirthCountry,Nationality,Updated)
    VALUES %s;
    """, [tuple(row) for row in df.values])

conn.commit()

In [85]:
cur.close()
conn.close()